# Train BPE Tokenizer
Train a custom BPE tokenizer

## Load and explore dataset obj

In [37]:
from datasets import load_dataset

In [38]:
input_data = load_dataset(path="TucanoBR/Tucano-SFT", cache_dir="../data", split='train')

In [39]:
type(input_data)

datasets.arrow_dataset.Dataset

In [40]:
print(dir(input_data))

['_TF_DATASET_REFS', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getitems__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slotnames__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_build_local_temp_path', '_check_index_is_initialized', '_data', '_estimate_nbytes', '_fast_select_column', '_fingerprint', '_format_columns', '_format_kwargs', '_format_type', '_generate_tables_from_cache_file', '_generate_tables_from_shards', '_get_cache_file_path', '_get_output_signature', '_getitem', '_indexes', '_indices', '_info', '_map_single', '_new_dataset_with_indices', '_output_all_columns', '_push_parquet_shards_to_hub', '_push_parquet

In [41]:
print(input_data)

Dataset({
    features: ['conversations', 'metadata'],
    num_rows: 679609
})


In [42]:
input_data[0]

{'conversations': [{'content': 'O que acontece a seguir neste parágrafo?\n\nEla então esfrega uma agulha em uma bola de algodão, em seguida, empurrando-a para um lápis e enrolando o fio em torno dela. Ela então segura uma caixa de um produto e, em seguida, despejando vários líquidos em uma tigela.\nEscolha a sua resposta de: A. adiciona panela e agita o produto em um moedor. B. belisca o fio para estilizar um cigarro e, em seguida, vai embora. C., em seguida, mergulha a agulha em tinta e usando o lápis para desenhar um design em sua perna, esfregando-o com um pano no final. D. começa a estilizar seu cabelo e corta várias vezes antes de separar as extremidades dele para mostrar o penteado que ela criou.',
   'role': 'user'},
  {'content': 'C. Ela então mergulha a agulha em tinta e usando o lápis para desenhar um design em sua perna, esfregando-o com um pano no final. Nesta opção, ela continua o processo de usar a agulha, lápis e fio, que está mais relacionada com o que ela estava fazend

## Prepare dataset for training

In [43]:
# Define a function to extract content from conversations
def extract_content(row):
    # Extract all "content" values from the "conversations" column
    contents_list = [document["content"] for document in row["conversations"] if "content" in document]
    content_texts = ''.join(contents_list)
    return {"content_texts": content_texts}

In [44]:
# create a new dataset with only text content
dataset = input_data.map(extract_content).select_columns('content_texts')

In [45]:
print(dataset)

Dataset({
    features: ['content_texts'],
    num_rows: 679609
})


In [46]:
dataset[0]

{'content_texts': 'O que acontece a seguir neste parágrafo?\n\nEla então esfrega uma agulha em uma bola de algodão, em seguida, empurrando-a para um lápis e enrolando o fio em torno dela. Ela então segura uma caixa de um produto e, em seguida, despejando vários líquidos em uma tigela.\nEscolha a sua resposta de: A. adiciona panela e agita o produto em um moedor. B. belisca o fio para estilizar um cigarro e, em seguida, vai embora. C., em seguida, mergulha a agulha em tinta e usando o lápis para desenhar um design em sua perna, esfregando-o com um pano no final. D. começa a estilizar seu cabelo e corta várias vezes antes de separar as extremidades dele para mostrar o penteado que ela criou.C. Ela então mergulha a agulha em tinta e usando o lápis para desenhar um design em sua perna, esfregando-o com um pano no final. Nesta opção, ela continua o processo de usar a agulha, lápis e fio, que está mais relacionada com o que ela estava fazendo na frase anterior.'}

## First Train

In [47]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers

In [73]:
# Create iterator for content_texts
def content_iterator(dataset):
    for row in dataset:
       yield row["content_texts"]

In [74]:
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [75]:
# Define trainer
trainer = trainers.BpeTrainer(
    vocab_size=60000,
    special_tokens=["[UNK]"],
    min_frequency=2,
    show_progress=True
)

In [76]:
%%time
tokenizer.train_from_iterator(content_iterator(dataset), trainer=trainer)

CPU times: total: 5min 59s
Wall time: 1min 46s


In [77]:
# Save tokenizer
tokenizer.save("portuguese_bpe_tokenizer_7.json")

## Explorer trained tokenizer

In [78]:
encoded = tokenizer.encode("Olá, como você está?")
print(encoded.ids)

[11744, 13, 2005, 2098, 2086, 32]


In [79]:
tokens_per_line = 20
total_tokens = tokenizer.get_vocab_size()
total_lines = int(total_tokens / tokens_per_line)
column_width = 6
print(f'Vocabulary size: {total_tokens}')

Vocabulary size: 60000


In [80]:
for i in range(0, total_lines):
    token_ids = [(tokens_per_line * i ) + id for id in range(0,tokens_per_line)]
    line = '|'.join(f"{token_id:>{column_width}}" for token_id in token_ids)
    print(line)
    
    tokens = [tokenizer.decode([token_id]) for token_id in token_ids]
    line = '|'.join(f"{token:>{column_width}}" for token in tokens)
    print(line, "\n")

     0|     1|     2|     3|     4|     5|     6|     7|     8|     9|    10|    11|    12|    13|    14|    15|    16|    17|    18|    19
      |     |     !|     "|     #|     $|     %|     &|     '|     (|     )|     *|     +|     ,|     -|     .|     /|     0|     1|     2 

    20|    21|    22|    23|    24|    25|    26|    27|    28|    29|    30|    31|    32|    33|    34|    35|    36|    37|    38|    39
     3|     4|     5|     6|     7|     8|     9|     :|     ;|     <|     =|     >|     ?|     @|     A|     B|     C|     D|     E|     F 

    40|    41|    42|    43|    44|    45|    46|    47|    48|    49|    50|    51|    52|    53|    54|    55|    56|    57|    58|    59
     G|     H|     I|     J|     K|     L|     M|     N|     O|     P|     Q|     R|     S|     T|     U|     V|     W|     X|     Y|     Z 

    60|    61|    62|    63|    64|    65|    66|    67|    68|    69|    70|    71|    72|    73|    74|    75|    76|    77|    78|    79
     [|     \|

## Clean out unwanted characters

In [81]:
remove_chars = []

init_id = 177
end_id = 1913
for i in range(init_id, end_id):
    remove_chars.append(tokenizer.decode([i]))

print(remove_chars)

['ü', 'ý', 'ā', 'ă', 'ą', 'Ć', 'ć', 'Ċ', 'Č', 'č', 'ď', 'Đ', 'đ', 'Ē', 'ē', 'ė', 'ę', 'ě', 'ğ', 'ģ', 'ĩ', 'ī', 'ĭ', 'İ', 'ı', 'Ľ', 'Ł', 'ł', 'ń', 'ņ', 'ň', 'Ō', 'ō', 'ő', 'œ', 'Ř', 'ř', 'Ś', 'ś', 'Ş', 'ş', 'Š', 'š', 'ţ', 'ũ', 'ū', 'ů', 'ų', 'ŷ', 'ź', 'Ż', 'ż', 'Ž', 'ž', 'ƒ', 'ơ', 'ư', 'ǎ', 'ǐ', 'ǒ', 'ǔ', 'ȗ', 'ș', 'ț', 'ȳ', 'ɑ', 'ɔ', 'ə', 'ʃ', 'ʇ', 'ʊ', 'ʻ', 'ʿ', 'ˆ', 'ˈ', '˚', '̀', '́', '̂', '̃', '̇', '̪', 'Α', 'Δ', 'Λ', 'Μ', 'Ρ', 'Σ', 'Τ', 'Χ', 'Ω', 'ά', 'έ', 'ί', 'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'π', 'ρ', 'ς', 'σ', 'τ', 'υ', 'φ', 'ψ', 'ω', 'ό', 'А', 'Б', 'В', 'К', 'Л', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'х', 'ч', 'ш', 'щ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'Ӧ', '،', 'ء', 'أ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ع', 'غ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي', 'ً', 'َ', 'ُ', 'ِ', 'ّ',

In [82]:
def remove_row(row, unwanted_chars):
    content = row['content_texts']

    if content is None:
        return True  # Keep rows with None content

    return not any(char in content for char in unwanted_chars)

In [83]:
dataset

Dataset({
    features: ['content_texts'],
    num_rows: 679609
})

In [84]:
filtered_dataset = dataset.filter(
    lambda example: remove_row(example, remove_chars),
    desc="Filtering out rows with unwanted characters"
)

In [85]:
filtered_dataset

Dataset({
    features: ['content_texts'],
    num_rows: 657685
})

## Second train
With filtered out row with unwanted characters

In [86]:
# Define new trainer
trainer = trainers.BpeTrainer(
    vocab_size=60000,
    special_tokens=["[UNK]"],
    min_frequency=3,
    show_progress=True
)

In [87]:
%%time
tokenizer.train_from_iterator(content_iterator(filtered_dataset), trainer=trainer)

CPU times: total: 5min 44s
Wall time: 1min 39s


In [88]:
encoded = tokenizer.encode("Olá, como você está?")
print(encoded.ids)

[11744, 13, 2005, 2098, 2086, 32]


In [89]:
tokens_per_line = 20
total_tokens = tokenizer.get_vocab_size()
total_lines = int(total_tokens / tokens_per_line)
column_width = 6
print(f'Vocabulary size: {total_tokens}')

Vocabulary size: 60000


In [90]:
for i in range(0, total_lines):
    token_ids = [(tokens_per_line * i ) + id for id in range(0,tokens_per_line)]
    line = '|'.join(f"{token_id:>{column_width}}" for token_id in token_ids)
    print(line)
    
    tokens = [tokenizer.decode([token_id]) for token_id in token_ids]
    line = '|'.join(f"{token:>{column_width}}" for token in tokens)
    print(line, "\n")

     0|     1|     2|     3|     4|     5|     6|     7|     8|     9|    10|    11|    12|    13|    14|    15|    16|    17|    18|    19
      |     |     !|     "|     #|     $|     %|     &|     '|     (|     )|     *|     +|     ,|     -|     .|     /|     0|     1|     2 

    20|    21|    22|    23|    24|    25|    26|    27|    28|    29|    30|    31|    32|    33|    34|    35|    36|    37|    38|    39
     3|     4|     5|     6|     7|     8|     9|     :|     ;|     <|     =|     >|     ?|     @|     A|     B|     C|     D|     E|     F 

    40|    41|    42|    43|    44|    45|    46|    47|    48|    49|    50|    51|    52|    53|    54|    55|    56|    57|    58|    59
     G|     H|     I|     J|     K|     L|     M|     N|     O|     P|     Q|     R|     S|     T|     U|     V|     W|     X|     Y|     Z 

    60|    61|    62|    63|    64|    65|    66|    67|    68|    69|    70|    71|    72|    73|    74|    75|    76|    77|    78|    79
     [|     \|

In [91]:
# Save tokenizer
tokenizer.save("portuguese_bpe_tokenizer_8.json")

## Third train

In [92]:
seed = 8483
num_samples = 200000
prepared_data = filtered_dataset.shuffle(seed).select(range(num_samples))

In [93]:
%%time
tokenizer.train_from_iterator(content_iterator(prepared_data), trainer=trainer)

CPU times: total: 1min 52s
Wall time: 31.6 s


In [94]:
encoded = tokenizer.encode("Olá, como você está?")
print(encoded.ids)

[11744, 13, 2005, 2098, 2086, 32]


In [95]:
tokens_per_line = 20
total_tokens = tokenizer.get_vocab_size()
total_lines = int(total_tokens / tokens_per_line)
column_width = 6
print(f'Vocabulary size: {total_tokens}')

Vocabulary size: 60000


In [96]:
for i in range(0, total_lines):
    token_ids = [(tokens_per_line * i ) + id for id in range(0,tokens_per_line)]
    line = '|'.join(f"{token_id:>{column_width}}" for token_id in token_ids)
    print(line)
    
    tokens = [tokenizer.decode([token_id]) for token_id in token_ids]
    line = '|'.join(f"{token:>{column_width}}" for token in tokens)
    print(line, "\n")

     0|     1|     2|     3|     4|     5|     6|     7|     8|     9|    10|    11|    12|    13|    14|    15|    16|    17|    18|    19
      |     |     !|     "|     #|     $|     %|     &|     '|     (|     )|     *|     +|     ,|     -|     .|     /|     0|     1|     2 

    20|    21|    22|    23|    24|    25|    26|    27|    28|    29|    30|    31|    32|    33|    34|    35|    36|    37|    38|    39
     3|     4|     5|     6|     7|     8|     9|     :|     ;|     <|     =|     >|     ?|     @|     A|     B|     C|     D|     E|     F 

    40|    41|    42|    43|    44|    45|    46|    47|    48|    49|    50|    51|    52|    53|    54|    55|    56|    57|    58|    59
     G|     H|     I|     J|     K|     L|     M|     N|     O|     P|     Q|     R|     S|     T|     U|     V|     W|     X|     Y|     Z 

    60|    61|    62|    63|    64|    65|    66|    67|    68|    69|    70|    71|    72|    73|    74|    75|    76|    77|    78|    79
     [|     \|

In [97]:
# Save tokenizer
tokenizer.save("portuguese_bpe_tokenizer_9.json")

Tokenizers parameters:
| Name | Vocab Size | sample size |
|------|------------|-------------|
| 1    | 30,000     | 679,609     |
| 2    | 30,000     | 657,685     |
| 3    | 30,000     | 200,000     |
| 4    | 15,000     | 679,609     |
| 5    | 15,000     | 657,685     |
| 6    | 15,000     | 200,000     |
| 7    | 60,000     | 679,609     |
| 8    | 60,000     | 657,685     |
| 9    | 60,000     | 200,000     |
